## Correcting points where p_display='true' to p_display="True"
Due to an error in the telegraf configuration, some points (around 4 for each sensor) were inserted with the `p_display` tag set to `'true'` rather than `'True'`. This corrects this by searching for those points, reinserting them with the correct tag, and then deleting the points with the old tag.

**Note:** The password is not included in this notebook so that it can be commited to Github. Please fill in the empty string in the line below to include the admin password for the InfluxDB server.

In [7]:
from influxdb import InfluxDBClient

client = InfluxDBClient('aqdata.uksouth.cloudapp.azure.com', 8086, 'admin', '', 'aqdata', ssl=True) 
db_data = client.query('SELECT * FROM "ttn" WHERE "p_display" = \'true\' ')


/Users/robin/anaconda3/envs/breathingspaces/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [8]:
tag_keys = ['app_id',
'dev_id',
'hardware_serial',
'm_coding_rate',
'm_data_rate',
'm_frequency',
'p_corrected',
'p_display',
'p_version',
'port']

In [9]:
def float_or_none(x):
    if x is None:
        return None
    else:
        return float(x)

In [10]:
float_keys = ['m_latitude', 'm_longitude', 'p_dht22_hum', 'p_dht22_pid',
              'p_dht22_temp', 'p_hpma115s0_pid', 'p_humidity', 'p_int_temp', 'p_load1', 'p_load2', 'p_load3',
              'p_opcn2_pid', 'p_pms5003_pid', 'p_pms7003_pid', 'p_temperature']

In [11]:
data_to_write = []
for point in db_data.get_points():
    tags = {k: point[k] for k in tag_keys}
    tags['p_display'] = True
    
    fields = {k: point[k] for k in point.keys() if k not in tag_keys}
    del fields['time']
    
    for key in float_keys:
        fields[key] = float_or_none(fields[key])
    
    new_point = {'measurement': 'ttn',
                  'tags': tags,
                  'time': point['time'],
                  'fields': fields,
                  }
    data_to_write.append(new_point)

In [13]:
len(data_to_write)

68

In [14]:
data_to_write[0]

{'measurement': 'ttn',
 'tags': {'app_id': 'aq-deployment',
  'dev_id': 'nesta-11',
  'hardware_serial': '00B390EDA54CF086',
  'm_coding_rate': '4/5',
  'm_data_rate': 'SF10BW125',
  'm_frequency': '867.3',
  'p_corrected': False,
  'p_display': True,
  'p_version': '2',
  'port': '1'},
 'time': '2020-02-04T12:00:06.185397007Z',
 'fields': {'airsensa0001': None,
  'aql-cowes-1': None,
  'counter': 8828,
  'csn-mtcdtip-003b02': None,
  'csn-mtcdtip-003b03': None,
  'eui-0000024b080601ef': None,
  'eui-0000024b0b030e6d': None,
  'eui-0000024b0b030f66': None,
  'eui-0000024b0b030f67': None,
  'eui-0000024b0b031c97': None,
  'eui-00800000a0001b7a': None,
  'eui-00800000a0004dac': None,
  'eui-00800000a0004ef2': None,
  'eui-58a0cbfffe8004f1': None,
  'eui-647fdafffe005203': None,
  'eui-7276ff002e0627e1': None,
  'eui-7276ff002e062804': None,
  'eui-7276ff002e062a15': None,
  'eui-7276fffffe010280': None,
  'eui-7276fffffe01028c': -111,
  'eui-7276fffffe010292': -135,
  'eui-7276fffffe0103

In [15]:
client.write_points(data_to_write)

/Users/robin/anaconda3/envs/breathingspaces/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


True

In [16]:
client.query("drop series from ttn where p_display='true'")

/Users/robin/anaconda3/envs/breathingspaces/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


ResultSet({})